# 심층 신경망 Tensorflow 예제

* Tensorflow 환경 설치

In [ ]:
pip install setuptools==44.0.0 tensorflow==1.14.0 Cython contextlib2 matplotlib pillow lxml gast==0.2.2

# 코드 시작

1. 패키지 임포트

In [ ]:
# -*- coding: utf-8 -*-

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import random

2. mnist 데이터 다운로드

In [ ]:
mnist = input_data.read_data_sets("./temp/MNIST_data/", one_hot=True)

3. 하이퍼파라미터 세팅

In [ ]:
learning_rate = 0.01
epochs = 40    
minibatch = 128   
display_step = 1   
input_size = 784   
h1_size = 256
h2_size = 256
output_size = 10

4. 플레이스홀더 정의

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, output_size])

5. 3 레이어 신경망 모델 함수

In [ ]:
def ANN_fuction(x):
    W1 = tf.Variable(tf.random_normal(shape=[input_size, h1_size]))
    b1 = tf.Variable(tf.random_normal(shape=[h1_size]))
    H1_output = tf.nn.relu(tf.matmul(x,W1) + b1)
    W2 = tf.Variable(tf.random_normal(shape=[h1_size, h2_size]))
    b2 = tf.Variable(tf.random_normal(shape=[h2_size]))
    H2_output = tf.nn.relu(tf.matmul(H1_output,W2) + b2)
    W_output = tf.Variable(tf.random_normal(shape=[h2_size, output_size]))
    b_output = tf.Variable(tf.random_normal(shape=[output_size]))
    logits = tf.matmul(H2_output,W_output) + b_output

    return logits

6. 신경망 모델 함수 호출

In [ ]:
logits = ANN_fuction(x)

7. 손실함수와 최적화 함수

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2( logits=logits, labels=y))

train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

8. 세션 시작

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 훈련 시작
    for epoch in range(epochs):
        average_loss = 0.
        total_batch = int(mnist.train.num_examples/minibatch)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(minibatch)
            _, current_loss = sess.run([train_step, loss], feed_dict={x: batch_x, y: batch_y})
        average_loss += current_loss / total_batch
        
        # 훈련 상태(현재 에포크의 손실 값) 출력
        if epoch % display_step == 0:
            print("[Epoch] %d, [Loss] %f" % ((epoch+1), average_loss))

    # 모델 정확도 계산
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Accuracy] %f" % (accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})))
    
    # 랜덤 데이터 예측 테스트
    r = random.randint(0, mnist.test.num_examples - 1)
    print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
    print("Prediction: ", sess.run(tf.argmax(logits, 1), feed_dict={x: mnist.test.images[r:r + 1]}))    
    